Modes of correlation: Model (oracle, personal), Granularity (recipe level, user level, recipe flat, user flat, all recipes for all users), Surprise (95th percentile, max), Method of correlation (spearman, pearson)

Libraries

In [1]:
import os, pickle, numpy as np
from GloVex.evaluate_personalised import survey_reader
from scipy.stats import pearsonr, spearmanr
cwd = os.getcwd()

Helper functions

In [2]:
def averageing_fn(surp_list):
	_avg_ = []
	for each_ in surp_list:
		_avg_.append(reduce(lambda x, y: x + y, each_) / len(each_))
	return _avg_

Get the surprises of the users in the survey

In [3]:
# survey_fp = '/Volumes/GoogleDrive/My Drive/Live/PQE/Omar/personalized-surprise/qchef_surveydata'
# user_fam_dict, users_surp_ratings_dict = survey_reader(survey_fp)

GloVex/evaluate_personalised.py:308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  users_df[col_list_name] = (np.array(users_df.values.tolist()) / 5.0).tolist()


In [3]:
survey_reader_obj = survey_reader()
food_cuisine_survey_fn = cwd + '/personalized-surprise/Food and Cuisine Preferences Survey (Responses) - Form Responses 1'
# users_fam_dir, users_cuisine_pref, users_knowledge, users_surp_ratings, users_surp_pref = 
user_fam_dict, users_cuisine_pref, users_knowledge, users_surp_ratings_dict, users_surp_pref =\
	survey_reader_obj.read_survey(food_cuisine_survey_fn)

GloVex/evaluate_personalised.py:308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  users_df[col_list_name] = (np.array(users_df.values.tolist()) / 5.0).tolist()


Prepare array from the dict

In [4]:
# Take the familiarity scores from the dict into arrays
user_fam_arr = []
for each_user in user_fam_dict:
	user_fam_arr.append(user_fam_dict[each_user])
# Take the surprise scores from the dict into arrays
users_surp_ratings_arr = []
for each_user in users_surp_ratings_dict:
	users_surp_ratings_arr.append(users_surp_ratings_dict[each_user])

Transpose for recipe level comparison

In [5]:
users_surp_ratings_userlvl = users_surp_ratings_arr
users_surp_ratings_recipelvl = np.transpose(users_surp_ratings_arr).tolist()

Flatten the arrays in both dimenssions

In [6]:
users_surp_ratings_user_flat = [item for sublist in users_surp_ratings_userlvl for item in sublist]
users_surp_ratings_recipe_flat = [item for sublist in users_surp_ratings_recipelvl for item in sublist]

Divide the users into foodies and novices using the median of sum of familiarities

In [7]:
# Array holding all of the sums
sum_arr = []
# Iterate over the users' familiarities
for user_idx in user_fam_dict.keys():
	# Get the familiarity scores
	each_fam = user_fam_dict[user_idx]
	# Calculate the sum of familiarity for each user
	sum_fam = sum(each_fam)
	# Append the sum of familiarity of each user to the sum array
	sum_arr.append(sum_fam)
median_fam = np.median(sum_arr)
# Split the users into foodeis and novices using the median
foodie_fam_dict = {}
foodie_surp_ratings_dict = {}
novice_fam_dict = {}
novice_surp_ratings_dict = {}
# Iterate over the users' familiarities
for user_idx in user_fam_dict.keys():
	# Get the familiarity and surprise scores
	each_fam = user_fam_dict[user_idx]
	each_surp = users_surp_ratings_dict[user_idx]
	# Calculate the sum of familiarity for each user
	sum_fam = sum(each_fam)
	if sum_fam > median_fam:
		# print 'above median', each_fam
		foodie_fam_dict[user_idx] = each_fam
		foodie_surp_ratings_dict[user_idx] = each_surp
	else:
		# print 'below median', each_fam
		novice_fam_dict[user_idx] = each_fam
		novice_surp_ratings_dict[user_idx] = each_surp
# List the foodies and novices in two separate lists
foodie_users = foodie_fam_dict.keys()
novice_users = novice_fam_dict.keys()
# print 'foodie_fam_dict', foodie_fam_dict
# print 'foodie_surp_ratings_dict', foodie_surp_ratings_dict
# print 'novice_fam_dict', novice_fam_dict
# print 'novice_surp_ratings_dict', novice_surp_ratings_dict

Get the personal surprise estimates by the model

In [9]:
user_suprise_estimates_pickle_fn = cwd + '/GloVex/results/second_survey/user_suprise_estimates.pickle'
user_suprise_estimates_dict = pickle.load(open(user_suprise_estimates_pickle_fn,'rb'))

Get the 95th percentils and max surprise estimates

In [10]:
# Initialize surprise estimates for 95th percentile and max score
personalized_surp_estimates_95perc = []
personalized_surp_estimates_95perc_dict = {}
personalized_surp_estimates_max = []
personalized_surp_estimates_max_dict = {}
# Iterate over the user surprise estimeates dict
for each_user in user_suprise_estimates_dict.keys():
	# print 'each_user', each_user
	# print user_suprise_estimates_dict[each_user]
	# Initialize the inner arrays for 95th percentile and max score
	users_surp_estimates_95perc = []
	users_surp_estimates_max = []
	# Iterate over the surprises for each user's surprise estimates
	for recipe_idx, each_recipe in enumerate(user_suprise_estimates_dict[each_user]['recipes_surp']):
		# print 'recipe_idx', recipe_idx, user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile']
		# Get all of the ingredients combinations' surprise estimates
		ingr_comb_suprise_arr = [each_ingr_comb[2] for each_ingr_comb in 
			user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['surprise_cuisine']]
		# Get the 95th percentile
		user_suprise_95th_percentile = user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile']
		# print user_suprise_95th_percentile
		# Get the max
		user_suprise_max = max(ingr_comb_suprise_arr)
		# print user_suprise_max
		# Append the 95th percentile for the surprise estimates
		users_surp_estimates_95perc.append(user_suprise_95th_percentile)
		# Append the max score for the surprise estimates
		users_surp_estimates_max.append(user_suprise_max)
	# Append the user's surprise estimates to the personalized main 2D array (95th percentile and max score)
	personalized_surp_estimates_95perc.append(users_surp_estimates_95perc)
	personalized_surp_estimates_max.append(users_surp_estimates_max)
	# 
	personalized_surp_estimates_95perc_dict[each_user] = users_surp_estimates_95perc
	personalized_surp_estimates_max_dict[each_user] = users_surp_estimates_max
# print personalized_surp_estimates_95perc
# print personalized_surp_estimates_max
# print personalized_surp_estimates_95perc_dict
# print personalized_surp_estimates_max_dict

Transpose for recipe level comparison

In [11]:
personalized_surp_estimates_recipelvl_95perc = np.transpose(personalized_surp_estimates_95perc).tolist()
personalized_surp_estimates_recipelvl_max = np.transpose(personalized_surp_estimates_max).tolist()

Flatten the arrays in both dimnessions

In [12]:
personalized_surp_estimates_userflat_95perc = [item for sublist in personalized_surp_estimates_95perc for item in sublist]
personalized_surp_estimates_userflat_max = [item for sublist in personalized_surp_estimates_max for item in sublist]
personalized_surp_estimates_recipeflat_95perc = [item for sublist in personalized_surp_estimates_recipelvl_95perc for item in sublist]
personalized_surp_estimates_recipeflat_max = [item for sublist in personalized_surp_estimates_recipelvl_max for item in sublist]

Separate the foodies from novices

In [13]:
# Get the foodies for the 95th percentile and the max
foodie_personalized_surp_estimates_95perc_dict = {each_: personalized_surp_estimates_95perc_dict[each_] 
	for each_ in personalized_surp_estimates_95perc_dict.keys() if each_ in foodie_surp_ratings_dict.keys()}
foodie_personalized_surp_estimates_max_dict = {each_: personalized_surp_estimates_max_dict[each_] 
	for each_ in personalized_surp_estimates_max_dict.keys() if each_ in foodie_surp_ratings_dict.keys()}
# Get the novices for the 95th percentile and the max
novice_personalized_surp_estimates_95perc_dict = {each_: personalized_surp_estimates_95perc_dict[each_] 
	for each_ in personalized_surp_estimates_95perc_dict.keys() if each_ in novice_surp_ratings_dict.keys()}
novice_personalized_surp_estimates_max_dict = {each_: personalized_surp_estimates_max_dict[each_] 
	for each_ in personalized_surp_estimates_max_dict.keys() if each_ in novice_surp_ratings_dict.keys()}

Transfer the foodies' and novices' into arrays using the same iterator's index for the ratings and estimates (at the same time to geuarantee synchronization)

In [14]:
# Foodies
foodie_surp_ratings = []
foodie_personalized_surp_estimates_95perc = []
foodie_personalized_surp_estimates_max = []
for each_foodie in foodie_users:
	foodie_surp_ratings.append(foodie_surp_ratings_dict[each_foodie])
	foodie_personalized_surp_estimates_95perc.append(foodie_personalized_surp_estimates_95perc_dict[each_foodie])
	foodie_personalized_surp_estimates_max.append(foodie_personalized_surp_estimates_max_dict[each_foodie])
# Novices
novice_surp_ratings = []
novice_personalized_surp_estimates_95perc = []
novice_personalized_surp_estimates_max = []
for each_novice in novice_users:
	novice_surp_ratings.append(novice_surp_ratings_dict[each_novice])
	novice_personalized_surp_estimates_95perc.append(novice_personalized_surp_estimates_95perc_dict[each_novice])
	novice_personalized_surp_estimates_max.append(novice_personalized_surp_estimates_max_dict[each_novice])

Foodies' and novices' user and recipe levels

In [15]:
# 
foodie_surp_ratings_userlvl = foodie_surp_ratings
foodie_surp_ratings_recipelvl = np.transpose(foodie_surp_ratings_userlvl).tolist()
# 
foodie_surp_ratings_user_flat = [item for sublist in foodie_surp_ratings_userlvl for item in sublist]
foodie_surp_ratings_recipe_flat = [item for sublist in foodie_surp_ratings_recipelvl for item in sublist]
# 
novice_surp_ratings_userlvl = novice_surp_ratings
novice_surp_ratings_recipelvl = np.transpose(novice_surp_ratings_userlvl).tolist()
# 
novice_surp_ratings_user_flat = [item for sublist in novice_surp_ratings_userlvl for item in sublist]
novice_surp_ratings_recipe_flat = [item for sublist in novice_surp_ratings_recipelvl for item in sublist]

Flatten the foodies' and novices' surprise ratings

In [16]:
# 
foodie_personalized_surp_estimates_userflat_95perc = [item for sublist in foodie_personalized_surp_estimates_95perc for item in sublist]
foodie_personalized_surp_estimates_userflat_max = [item for sublist in foodie_personalized_surp_estimates_max for item in sublist]
# 
foodie_personalized_surp_estimates_recipelvl_95perc = np.transpose(foodie_personalized_surp_estimates_95perc).tolist()
foodie_personalized_surp_estimates_recipeflat_95perc = [item for sublist in foodie_personalized_surp_estimates_recipelvl_95perc for item in sublist]
foodie_personalized_surp_estimates_recipelvl_max = np.transpose(foodie_personalized_surp_estimates_max).tolist()
foodie_personalized_surp_estimates_recipeflat_max = [item for sublist in foodie_personalized_surp_estimates_recipelvl_max for item in sublist]
# 
novice_personalized_surp_estimates_userflat_95perc = [item for sublist in novice_personalized_surp_estimates_95perc for item in sublist]
novice_personalized_surp_estimates_userflat_max = [item for sublist in novice_personalized_surp_estimates_max for item in sublist]
# 
novice_personalized_surp_estimates_recipelvl_95perc = np.transpose(novice_personalized_surp_estimates_95perc).tolist()
novice_personalized_surp_estimates_recipeflat_95perc = [item for sublist in novice_personalized_surp_estimates_recipelvl_95perc for item in sublist]
novice_personalized_surp_estimates_recipelvl_max = np.transpose(novice_personalized_surp_estimates_max).tolist()
novice_personalized_surp_estimates_recipeflat_max = [item for sublist in novice_personalized_surp_estimates_recipelvl_max for item in sublist]

Get the oracle surprise estimates by the model

In [17]:
oracle_suprise_estimates_pickle_fn = cwd + '/GloVex/results/second_survey/oracle_suprise_estimates.pickle'
oracle_suprise_estimates_dict = pickle.load(open(oracle_suprise_estimates_pickle_fn, 'rb'))

Get the oracle's surprise estimates (95th percentile and max score)

In [18]:
# Initialize arrays
oracle_surp_estimates_95perc = []
oracle_surp_estimates_max = []
# Iterate over the dict
for each_recipe in oracle_suprise_estimates_dict:
	# Get all of the ingredients combinations' surprise estimates
	oracle_ingr_surp_estimates = [each_ingr_comb[2] for each_ingr_comb in oracle_suprise_estimates_dict[each_recipe]['surprise_cuisine']]
	# Append the 95th_percentile surprise estimate
	oracle_surp_estimates_95perc.append(oracle_suprise_estimates_dict[each_recipe]['95th_percentile'])
	# Append the max score surprise estimate
	oracle_surp_estimates_max.append(max(oracle_ingr_surp_estimates))
# oracle_surp_estimates_95perc
# oracle_surp_estimates_max

Get the averages on the recipe level

In [19]:
# All users ratings
users_surp_ratings_recipelvl_avg = averageing_fn(users_surp_ratings_recipelvl)
# Foodies ratings
foodie_surp_ratings_recipelvl_avg = averageing_fn(foodie_surp_ratings_recipelvl)
# Novices ratings
novice_surp_ratings_recipelvl_avg = averageing_fn(novice_surp_ratings_recipelvl)
# All users estimates 95th perc
personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(personalized_surp_estimates_recipelvl_95perc)
# All users estimates max
personalized_surp_estimates_recipelvl_max_avg = averageing_fn(personalized_surp_estimates_recipelvl_max)
# Foodies estimates 95th perc
foodie_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_95perc)
# Foodies estimates max
foodie_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_max)
# Novices estimates 95th perc
novice_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_95perc)
# Novices estimates max
novice_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_max)

Find correlation between surprise scores

Function for calculating correlation per user

In [20]:
# Function for calculating the correlation per user
def corr_per_user(oracle_surp_estimates, personalized_surp_estimates, users_surp_ratings_userlvl):
	for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates, users_surp_ratings_userlvl)):
		print user_idx
		print each_personal_score
		print each_users_rating
		print 'oracle_surp_estimates: spearmanr', spearmanr(oracle_surp_estimates, each_users_rating)
		print 'oracle_surp_estimates: pearsonr', pearsonr(oracle_surp_estimates, each_users_rating)
		print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
		print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

Mode: Oracle + Personalized, per user (default for oracle), 95th percentile and max

In [24]:
corr_per_user(oracle_surp_estimates_95perc, personalized_surp_estimates_95perc, users_surp_ratings_userlvl)

0
[-0.808891144861552, 0.4244098733381895, 0.6191976828781225, -0.4858618153793174, -0.7938680245575195, 0.6676737506543786, 0.3640200993618725, 1.8612318656397748, 0.8668828612549512, 2.982957096945315, 0.928807181852486, 1.0224307014397156, -0.5090925090533707, -0.7183846862950984, -0.08295239083091413, 0.45092412966460726]
[0.2, 0.4, 0.6, 0.6, 0.6, 0.2, 0.8, 0.6, 0.2, 0.2, 0.8, 0.4, 0.8, 0.4, 0.4, 0.4]
oracle_surp_estimates: spearmanr (0.27675047427104615, 0.29943300840619014)
oracle_surp_estimates: pearsonr (0.22176961975654536, 0.40910225086119467)
each_personal_score: spearmanr (-0.14597827214296943, 0.5895753585145861)
each_personal_score: pearsonr (-0.20826695620233254, 0.4389085157399095)
1
[-0.8579138506867598, 0.20444539759961733, 0.5416134164487005, -0.678679034145001, -0.6740641466321797, 0.976342777279018, -0.06633125857010472, 1.5816739393576649, 0.9699345962224303, 1.9531099154662834, 1.3604580410743516, 1.1833730757187704, -0.8388023236221633, -0.9163224542420301, -0.3

In [25]:
corr_per_user(oracle_surp_estimates_max, personalized_surp_estimates_max, users_surp_ratings_userlvl)

0
[-0.645909534422802, 1.754169541261711, 0.9289949776556362, -0.047681217275413725, -0.24229580422126176, 1.4218363740022382, 0.5828693132849753, 2.1796449772962347, 1.365043055123342, 3.5163794350142807, 1.5468550942882462, 1.7932786145276052, 0.36411329618529154, 0.12754379072993363, 0.6456864137116117, 1.436627340249848]
[0.2, 0.4, 0.6, 0.6, 0.6, 0.2, 0.8, 0.6, 0.2, 0.2, 0.8, 0.4, 0.8, 0.4, 0.4, 0.4]
oracle_surp_estimates: spearmanr (0.20528194520105075, 0.44564350642318085)
oracle_surp_estimates: pearsonr (0.07654549584540864, 0.7781289087839355)
each_personal_score: spearmanr (-0.17030798416679765, 0.5282983847872502)
each_personal_score: pearsonr (-0.24088939052829178, 0.3687944977098108)
1
[-0.7511908024085738, 1.75408100530445, 0.8583625637097416, -0.4316424764441391, -0.054689359484461374, 1.295764969719904, 0.5713497181840704, 2.216365902533948, 1.9216061198328513, 3.4875109820633012, 1.5112491923236058, 2.686375014234605, 0.0351733735148565, 0.2928012697132954, 0.3456328720

In [26]:
# 
corr_per_user(oracle_surp_estimates_95perc, foodie_personalized_surp_estimates_95perc, foodie_surp_ratings_userlvl)

0
[-0.808891144861552, 0.4244098733381895, 0.6191976828781225, -0.4858618153793174, -0.7938680245575195, 0.6676737506543786, 0.3640200993618725, 1.8612318656397748, 0.8668828612549512, 2.982957096945315, 0.928807181852486, 1.0224307014397156, -0.5090925090533707, -0.7183846862950984, -0.08295239083091413, 0.45092412966460726]
[0.2, 0.4, 0.6, 0.6, 0.6, 0.2, 0.8, 0.6, 0.2, 0.2, 0.8, 0.4, 0.8, 0.4, 0.4, 0.4]
oracle_surp_estimates: spearmanr (0.27675047427104615, 0.29943300840619014)
oracle_surp_estimates: pearsonr (0.22176961975654536, 0.40910225086119467)
each_personal_score: spearmanr (-0.14597827214296943, 0.5895753585145861)
each_personal_score: pearsonr (-0.20826695620233254, 0.4389085157399095)
1
[-0.3459550359581849, 0.8935127541405411, 1.1262960122234589, -0.03325671708650724, -0.221399025400082, 1.1655346948780239, 0.6700963081428127, 2.443788267154735, 1.3031748348629537, 3.263321967524252, 1.5591887681004204, 1.5979392813162725, -0.11218024812145189, -0.3410894584702777, 0.3045

In [27]:
corr_per_user(oracle_surp_estimates_max, foodie_personalized_surp_estimates_max, foodie_surp_ratings_userlvl)

0
[-0.645909534422802, 1.754169541261711, 0.9289949776556362, -0.047681217275413725, -0.24229580422126176, 1.4218363740022382, 0.5828693132849753, 2.1796449772962347, 1.365043055123342, 3.5163794350142807, 1.5468550942882462, 1.7932786145276052, 0.36411329618529154, 0.12754379072993363, 0.6456864137116117, 1.436627340249848]
[0.2, 0.4, 0.6, 0.6, 0.6, 0.2, 0.8, 0.6, 0.2, 0.2, 0.8, 0.4, 0.8, 0.4, 0.4, 0.4]
oracle_surp_estimates: spearmanr (0.20528194520105075, 0.44564350642318085)
oracle_surp_estimates: pearsonr (0.07654549584540864, 0.7781289087839355)
each_personal_score: spearmanr (-0.17030798416679765, 0.5282983847872502)
each_personal_score: pearsonr (-0.24088939052829178, 0.3687944977098108)
1
[-0.21866208216576719, 2.34148307038941, 1.35696406990715, 0.2865514147219877, 0.298180368848074, 1.9170490501471316, 1.0499458132016728, 2.557024490365672, 1.8018097560047237, 3.802050823220204, 2.0653722137479273, 2.8839821659617195, 0.8462902713398351, 0.7778077335212992, 1.065033527454583

In [28]:
# 
corr_per_user(oracle_surp_estimates_95perc, novice_personalized_surp_estimates_95perc, novice_surp_ratings_userlvl)

0
[-0.8579138506867598, 0.20444539759961733, 0.5416134164487005, -0.678679034145001, -0.6740641466321797, 0.976342777279018, -0.06633125857010472, 1.5816739393576649, 0.9699345962224303, 1.9531099154662834, 1.3604580410743516, 1.1833730757187704, -0.8388023236221633, -0.9163224542420301, -0.33078313811862814, 0.14381614642670235]
[0.2, 0.4, 0.2, 0.4, 0.2, 0.4, 0.8, 0.8, 0.6, 1.0, 0.6, 0.2, 1.0, 0.8, 0.2, 0.4]
oracle_surp_estimates: spearmanr (-0.13751307773195456, 0.6115535495124806)
oracle_surp_estimates: pearsonr (-0.2705152514237287, 0.3108962719644921)
each_personal_score: spearmanr (0.167735732178538, 0.534637683752339)
each_personal_score: pearsonr (0.21167260039308286, 0.43128838194784336)
1
[-1.323050068731493, 0.018625498068343356, -0.14923846353459005, -1.1335332971339507, -1.2837691845333452, 0.3516373147801758, -0.12280052287850757, 1.3621092994545463, 0.3384197334295405, 1.460179356595374, 0.7408822310048113, 0.6629049373975117, -0.9572168432698471, -1.2438565819840852, -0

In [29]:
corr_per_user(oracle_surp_estimates_max, novice_personalized_surp_estimates_max, novice_surp_ratings_userlvl)

0
[-0.7511908024085738, 1.75408100530445, 0.8583625637097416, -0.4316424764441391, -0.054689359484461374, 1.295764969719904, 0.5713497181840704, 2.216365902533948, 1.9216061198328513, 3.4875109820633012, 1.5112491923236058, 2.686375014234605, 0.0351733735148565, 0.2928012697132954, 0.3456328720592707, 1.4385570411971176]
[0.2, 0.4, 0.2, 0.4, 0.2, 0.4, 0.8, 0.8, 0.6, 1.0, 0.6, 0.2, 1.0, 0.8, 0.2, 0.4]
oracle_surp_estimates: spearmanr (-0.08613456517276276, 0.751107031864232)
oracle_surp_estimates: pearsonr (-0.2850323044873729, 0.28460031053529283)
each_personal_score: spearmanr (0.27351502274157996, 0.3053496205740897)
each_personal_score: pearsonr (0.2885673343071367, 0.2784061456051312)
1
[-1.1807569618674303, 1.0154311234978868, 0.19462277623965327, -0.7811524532662275, -0.3690592981324412, 1.3479808565793918, 0.1788578577382262, 2.1778290924042345, 0.724201724849504, 2.9853145344902514, 1.2837055117760037, 2.4697534119285454, -0.1881372832562298, -0.1685067288912359, -0.00630753898

Function for calculating correlation per recipe

In [30]:
def corr_per_recipe(personalized_surp_estimates_recipelvl, users_surp_ratings_recipelvl):
	for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_recipelvl, users_surp_ratings_recipelvl)):
		print user_idx
		print each_personal_score
		print each_users_rating
		print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
		print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

Mode: Personalized, per recipes, 95th percentile and max

In [31]:
corr_per_recipe(personalized_surp_estimates_recipelvl_95perc, users_surp_ratings_recipelvl)

0
[-0.808891144861552, -0.8579138506867598, -1.323050068731493, -1.105187370425998, -1.8003904065458793, -0.3459550359581849, -0.8928595997274167, -0.4733734640538326, -0.9050020380379356, -0.6809027950601862, -0.4784469459906203, -0.4899185933919, -1.1332364631972884, -0.5305336649472714, -1.2488265185690417, -0.08575625593110567, -1.087577549208129, -1.346906218897486, -0.8225481992496919, -1.1541451653702, -0.4784469459906203, -0.6765589579491698, -0.8630914093997858, -0.5725580649516944, -0.6083859354054494, -0.49803139172840966, -1.9739898677575627, -0.8333376487983186, -0.4922085596637272, -0.6078448548319896, -1.1436002973885981, -1.0840651245371324, -0.3664299309421276, -0.08575625593110567, -0.49228822427517316, -1.0422320019366693, -1.4167778843340408, -1.238856504899771, -1.2332596535277687, -1.0213060099145774, -1.1501699286020488, -0.4809337991733911, -0.7359172253576587, -0.9631151646742078, -0.694454588287461, -1.0213060099145774, -0.9477074823008016, -0.4784469459906203

In [32]:
corr_per_recipe(personalized_surp_estimates_recipelvl_max, users_surp_ratings_recipelvl)

0
[-0.645909534422802, -0.7511908024085738, -1.1807569618674303, -0.975876654506434, -1.6561721690883813, -0.21866208216576719, -0.7735267858651993, -0.31768558431156557, -0.8026019190799041, -0.554792658569005, -0.35190697606280225, -0.3694751754540095, -1.0508061859947264, -0.40813148080934936, -1.1163243703256402, 0.040314787114061904, -0.978934632741114, -1.226393526244626, -0.6970571182748607, -1.0398659232532839, -0.35190697606280225, -0.5569561712242918, -0.7490966735778091, -0.4493467936105916, -0.48739419980734283, -0.36453755033665175, -1.787889193089342, -0.735531435243556, -0.36693656579665307, -0.48286577785453516, -1.078274585737539, -0.9771568084362078, -0.24243602373104428, 0.040314787114061904, -0.38568142809225237, -0.918232394037593, -1.3096317818435939, -1.1228836442552457, -1.1202515913262798, -0.9031146712669403, -1.0152667305256902, -0.36613789876483355, -0.619337738291691, -0.8358506209587945, -0.5807952238300578, -0.9031146712669403, -0.8310891320184167, -0.351

In [33]:
# 
corr_per_recipe(foodie_personalized_surp_estimates_recipelvl_95perc, foodie_surp_ratings_recipelvl)

0
[-0.808891144861552, -0.3459550359581849, -0.4733734640538326, -0.6809027950601862, -0.4784469459906203, -0.4899185933919, -0.5305336649472714, -0.08575625593110567, -0.8225481992496919, -0.4784469459906203, -0.6765589579491698, -0.5725580649516944, -0.6083859354054494, -0.49803139172840966, -0.8333376487983186, -0.4922085596637272, -0.6078448548319896, -0.3664299309421276, -0.08575625593110567, -0.49228822427517316, -0.4809337991733911, -0.7359172253576587, -0.694454588287461, -0.4784469459906203, -0.33831260659741236, -0.44255214531911247, -0.7182155595665061, -0.340574141386641, -0.6005557539899525, -0.8316605605983611, -0.5026227306122393, -0.7182155595665061, -0.4872163584219462, -0.08575625593110567, -0.48154749525649654, -0.8316605605983611, -0.5530439203900219, -0.08575625593110567, -0.6865879352938693, -0.6476685653033999, -0.3940122729334208, -0.505467090893644, -0.6372261536294535, -0.7024059955137189, -0.780661584314754, -0.41104544578988145, -0.41947046162083207, -0.3585

In [34]:
corr_per_recipe(foodie_personalized_surp_estimates_recipelvl_max, foodie_surp_ratings_recipelvl)

0
[-0.645909534422802, -0.21866208216576719, -0.31768558431156557, -0.554792658569005, -0.35190697606280225, -0.3694751754540095, -0.40813148080934936, 0.040314787114061904, -0.6970571182748607, -0.35190697606280225, -0.5569561712242918, -0.4493467936105916, -0.48739419980734283, -0.36453755033665175, -0.735531435243556, -0.36693656579665307, -0.48286577785453516, -0.24243602373104428, 0.040314787114061904, -0.38568142809225237, -0.36613789876483355, -0.619337738291691, -0.5807952238300578, -0.35190697606280225, -0.20818470190136854, -0.33256526841890927, -0.6024585713797382, -0.22661663108823493, -0.46466953583044973, -0.6977641016740682, -0.38644881924153235, -0.6024585713797382, -0.3638749825472423, 0.040314787114061904, -0.3596182726403351, -0.6977641016740682, -0.43315436674478, 0.040314787114061904, -0.5809074409273256, -0.5306179139950591, -0.29146717076303624, -0.379119277573142, -0.5086967828320478, -0.5729776949572063, -0.6634931289137862, -0.2820311001233724, -0.308136165945

In [35]:
# 
corr_per_recipe(novice_personalized_surp_estimates_recipelvl_95perc, novice_surp_ratings_recipelvl)

0
[-0.8579138506867598, -1.323050068731493, -1.105187370425998, -1.8003904065458793, -0.8928595997274167, -0.9050020380379356, -1.1332364631972884, -1.2488265185690417, -1.087577549208129, -1.346906218897486, -1.1541451653702, -0.8630914093997858, -1.9739898677575627, -1.1436002973885981, -1.0840651245371324, -1.0422320019366693, -1.4167778843340408, -1.238856504899771, -1.2332596535277687, -1.0213060099145774, -1.1501699286020488, -0.9631151646742078, -1.0213060099145774, -0.9477074823008016, -0.9206672440782953, -1.1399554913461702, -1.1841074845702768, -1.1501914349365818, -0.9122772710694362, -1.4154335968072769, -1.127843414215059, -0.9055446439993966, -1.0213060099145774, -1.251818204954261, -0.9507763217351138, -1.2719033783385814, -1.5552925518582679, -0.9907717335542439, -1.4443336622000817, -0.9892068018594301, -1.3186960943490902, -0.9385768185691926, -1.5183617311917041, -0.9405387113151895, -0.9055446439993966, -1.2734096899729463, -0.9768879865413278, -1.0213060099145774,

In [32]:
# 
corr_per_recipe(novice_personalized_surp_estimates_recipelvl_max, novice_surp_ratings_recipelvl)

0
[0.8217086324646126, 0.9499809219691805, 0.4004501418761413, 0.3531153558762443, 0.5631289475364123, 0.7542831926744691, 0.15243059451739382, 0.5690688686756857, 0.48790048028225025, 0.44966594799104953, 0.5634013604948671, 0.5724651120690639, 0.559542059506078, 0.49291495555769343, 0.2742870569656812, 0.5948570833783334, 0.5488822103271285, 0.9112649398540451, 0.760236127571921, 0.5963713040722619, 0.6666510301848653, 0.3497411820031651, 0.6045374508395838, 0.6711194619383872, 0.3929374208760536, 0.21813644804663868, 0.36429534563143423, 0.48527489739537805, 0.7185216313617284, 0.5920639866085953, 0.6022290987172171, 0.5753534515678806, 0.5945107254379447, 0.5297502960793254, 0.2760169495862016, -0.07044176741913688, -0.002351258844356851, 0.37415244021656846, 0.3929374208760536, 0.09717144358406316, 0.5612865391584673, -1.2987107298720704, 0.760236127571921, 0.28328189976540574, -0.7089343014507407, 0.2640846535534821, 0.2927441208902306]
[1, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 

Get averages for the recipe level arrays

In [ ]:
# All users ratings
users_surp_ratings_recipelvl_avg = averageing_fn(users_surp_ratings_recipelvl)
# Foodies ratings
foodie_surp_ratings_recipelvl_avg = averageing_fn(foodie_surp_ratings_recipelvl)
# Novices ratings
novice_surp_ratings_recipelvl_avg = averageing_fn(novice_surp_ratings_recipelvl)
# All users estimates 95th perc
personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(personalized_surp_estimates_recipelvl_95perc)
# All users estimates max
personalized_surp_estimates_recipelvl_max_avg = averageing_fn(personalized_surp_estimates_recipelvl_max)
# Foodies estimates 95th perc
foodie_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_95perc)
# Foodies estimates max
foodie_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(foodie_personalized_surp_estimates_recipelvl_max)
# Novices estimates 95th perc
novice_personalized_surp_estimates_recipelvl_95perc_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_95perc)
# Novices estimates max
novice_personalized_surp_estimates_recipelvl_max_avg = averageing_fn(novice_personalized_surp_estimates_recipelvl_max)


Correlation of the average

In [37]:
# Correlating personalized surprise estimates of 95th percentile with users' ratings
print 'spearmanr', spearmanr(personalized_surp_estimates_recipelvl_95perc_avg, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipelvl_95perc_avg, users_surp_ratings_recipelvl_avg)

spearmanr (0.44117647058823534, 0.08715363626051081)
pearsonr (0.5967940248977555, 0.014664963428679288)


In [38]:
# Correlating personalized surprise estimates of max with users' ratings
print 'spearmanr', spearmanr(personalized_surp_estimates_recipelvl_max_avg, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipelvl_max_avg, users_surp_ratings_recipelvl_avg)

spearmanr (0.3970588235294118, 0.1278060090882619)
pearsonr (0.539401868400027, 0.031056184975671555)


In [39]:
# Correlating foodies personalized surprise estimates of 95th perc with users' ratings
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipelvl_95perc_avg, foodie_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipelvl_95perc_avg, foodie_surp_ratings_recipelvl_avg)

spearmanr (0.2352941176470588, 0.38035580118865675)
pearsonr (0.4200404424560477, 0.10526793316235446)


In [40]:
# Correlating foodies personalized surprise estimates of max with users' ratings
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipelvl_max_avg, foodie_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipelvl_max_avg, foodie_surp_ratings_recipelvl_avg)

spearmanr (0.15882352941176472, 0.5568612810855389)
pearsonr (0.36774100703331086, 0.1611171075837267)


In [41]:
# Correlating novices personalized surprise estimates of 95th perc with users' ratings
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)

spearmanr (0.5029411764705883, 0.047063140467273036)
pearsonr (0.6394275489530709, 0.007650792244687562)


In [42]:
# Correlating novices personalized surprise estimates of max with users' ratings
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipelvl_95perc_avg, novice_surp_ratings_recipelvl_avg)

spearmanr (0.5029411764705883, 0.047063140467273036)
pearsonr (0.6394275489530709, 0.007650792244687562)


In [43]:
# Correlating oracle estimates 95th perc with users' ratings
print 'spearmanr', spearmanr(oracle_surp_estimates_95perc, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(oracle_surp_estimates_95perc, users_surp_ratings_recipelvl_avg)

spearmanr (-0.08529411764705883, 0.7534650694747833)
pearsonr (-0.05297006744014085, 0.8455271306226584)


In [44]:
# Correlating oracle estimates max with users' ratings
print 'spearmanr', spearmanr(oracle_surp_estimates_max, users_surp_ratings_recipelvl_avg)
print 'pearsonr', pearsonr(oracle_surp_estimates_max, users_surp_ratings_recipelvl_avg)

spearmanr (-0.14705882352941177, 0.5867933937012053)
pearsonr (-0.10253156853905493, 0.7055375125083014)


Modes: Personalized, all users, user flat, 95th percentile/max score


In [45]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)

95th percentile
spearmanr (0.2028062720236212, 1.1131688858812368e-18)
pearsonr (0.19482095778393715, 2.4802032442122867e-17)
Max score
spearmanr (0.19243464956877035, 6.112609353363518e-17)
pearsonr (0.18830366251182723, 2.8340519474626573e-16)


In [46]:
# Foodies
print '95th percentile'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_userflat_95perc, foodie_surp_ratings_user_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_userflat_95perc, foodie_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_userflat_max, foodie_surp_ratings_user_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_userflat_max, foodie_surp_ratings_user_flat)
# Novices
print '95th percentile'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_userflat_95perc, novice_surp_ratings_user_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_userflat_95perc, novice_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_userflat_max, novice_surp_ratings_user_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_userflat_max, novice_surp_ratings_user_flat)

95th percentile
spearmanr (0.10670079772454726, 0.0012505576587291429)
pearsonr (0.09583954632608378, 0.0037675904238249616)
Max score
spearmanr (0.0948076201040413, 0.004161440031484037)
pearsonr (0.08718106336565644, 0.00843316963608015)
95th percentile
spearmanr (0.2589914030250032, 6.205921202955256e-16)
pearsonr (0.27028086083432606, 2.88713385053302e-17)
Max score
spearmanr (0.25676493621347846, 1.117040444322183e-15)
pearsonr (0.26034317542191987, 4.331402091782202e-16)


Modes: Personalized, all recipes, recipe flat, 95th percentile/max score

In [47]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)


95th percentile
spearmanr (0.2028062720236212, 1.1131688858812368e-18)
pearsonr (0.19482095778393713, 2.4802032442122867e-17)
Max score
spearmanr (0.19243464956877035, 6.112609353363518e-17)
pearsonr (0.18830366251182723, 2.8340519474626573e-16)


In [48]:
# Foodies
print '95th percentile'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipeflat_95perc, foodie_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipeflat_95perc, foodie_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(foodie_personalized_surp_estimates_recipeflat_max, foodie_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(foodie_personalized_surp_estimates_recipeflat_max, foodie_surp_ratings_recipe_flat)
# Novices
print '95th percentile'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipeflat_95perc, novice_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipeflat_95perc, novice_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(novice_personalized_surp_estimates_recipeflat_max, novice_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(novice_personalized_surp_estimates_recipeflat_max, novice_surp_ratings_recipe_flat)

95th percentile
spearmanr (0.10670079772454726, 0.0012505576587291429)
pearsonr (0.09583954632608377, 0.0037675904238249616)
Max score
spearmanr (0.0948076201040413, 0.004161440031484037)
pearsonr (0.08718106336565644, 0.00843316963608015)
95th percentile
spearmanr (0.2589914030250032, 6.205921202955256e-16)
pearsonr (0.270280860834326, 2.88713385053302e-17)
Max score
spearmanr (0.25676493621347846, 1.117040444322183e-15)
pearsonr (0.2603431754219199, 4.331402091782202e-16)


Replicate the oracle by the number of the users to be able to correlate with the users' ratings (the arrays have to be the same size)

In [49]:
oracle_surp_estimates_repeated_95perc = [oracle_surp_estimates_95perc] * len(users_surp_ratings_userlvl)
oracle_surp_estimates_repeated_max = [oracle_surp_estimates_max] * len(users_surp_ratings_userlvl)
oracle_surp_estimates_userflat_95perc = [item for sublist in oracle_surp_estimates_repeated_95perc for item in sublist]
oracle_surp_estimates_userflat_max = [item for sublist in oracle_surp_estimates_repeated_max for item in sublist]

In [50]:
print '95th percentile'
print 'spearmanr', spearmanr(oracle_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(oracle_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(oracle_surp_estimates_userflat_max, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(oracle_surp_estimates_userflat_max, users_surp_ratings_user_flat)

95th percentile
spearmanr (0.0271098641694744, 0.24306647115128582)
pearsonr (-0.01572330234920918, 0.4984274700349569)
Max score
spearmanr (0.009368518087882027, 0.6866944038905238)
pearsonr (-0.03043482725975433, 0.18999539418031658)
